In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from  skopt import BayesSearchCV
from sklearn.model_selection import train_test_split

In [2]:
development = pd.read_csv("./../../DSL_Winter_Project_2024/development.csv")
evaluation = pd.read_csv("./../../DSL_Winter_Project_2024/evaluation.csv")

In [3]:
outlier_column_index=[0, 7, 12, 15, 16, 17]
columns_to_drop=[]

for index in outlier_column_index:
    columns_to_drop.append('pmax[%s]' % index)
    columns_to_drop.append('negpmax[%s]' % index)
    columns_to_drop.append('tmax[%s]' % index)
    columns_to_drop.append('area[%s]' % index)
    columns_to_drop.append('rms[%s]' % index)

eva=evaluation.drop(columns=columns_to_drop)
eva_df=eva.drop(columns=["Id"])

df=development.drop(columns=columns_to_drop)
X=df.drop(columns=['x', 'y'])
y=df.loc[:,['x', 'y']]

rs=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rs)

In [4]:
parameters = {"n_estimators":[50,100], "max_depth":[50,100], "min_samples_split": [2,5,10], "max_features":[ "sqrt", "log2"], "random_state":[rs], "n_jobs":[-1] } 
bayes = BayesSearchCV(RandomForestRegressor(), search_spaces=parameters, scoring="r2", cv=5, n_iter=20, n_jobs=-1, random_state=rs)

In [5]:
np.int = int
bayes.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20, n_jobs=-1,
              random_state=42, scoring='r2',
              search_spaces={'max_depth': [50, 100],
                             'max_features': ['sqrt', 'log2'],
                             'min_samples_split': [2, 5, 10],
                             'n_estimators': [50, 100], 'n_jobs': [-1],
                             'random_state': [42]})

In [6]:
bayes.best_score_

0.9988475948978112

In [7]:
bayes.best_params_

OrderedDict([('max_depth', 50),
             ('max_features', 'sqrt'),
             ('min_samples_split', 2),
             ('n_estimators', 100),
             ('n_jobs', -1),
             ('random_state', 42)])

In [8]:
rfr = RandomForestRegressor(n_estimators=100, min_samples_split=2, max_features='sqrt', max_depth= 50, random_state=42, n_jobs=-1)
rfr.fit(X_train, y_train)
y_rfr=rfr.predict(X_test)
distance=np.mean(np.sqrt(np.sum((y_rfr - y_test) ** 2, axis=1)))
print(distance)

eva_y=rfr.predict(eva_df)

df = pd.DataFrame(eva_y, columns=['Id', 'Predicted'])
df['Predicted'] = df.apply(lambda row: f"{row['Id']:.1f}|{row['Predicted']:.1f}", axis=1)
df['Id'] = df.index
df.to_csv('outputbayes0.csv', index=False, sep=',')


4.576920443421794
